In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import multiprocessing
import os.path

# Downloading stopwords (just a mock step since we don't have internet access here)
# nltk.download('stopwords')
# nltk.download('punkt')

In [5]:
from joblib import Parallel, delayed
from tqdm import tqdm
import re

def preprocess_text(text):
    """Tokenize, lowercase, and remove stopwords from the text."""
    # remove punctuation
    clean_text = re.sub(r'[^a-z]', ' ', text.lower())
    tokens = word_tokenize(clean_text)
    tokens = [stemmer.stem(word) for word in tokens]
    tok_sent = [word for word in tokens if word not in stop_words and word.isalpha()]
    if len(tok_sent) == 0:
        print("Empty tokienized string: ", text)
        return ['and'] # use a stop word as special token
    return tok_sent #' '.join(tok_sent)

def multiprocess_preprocess_joblib(data, column_name):
    """Preprocess data using multiple processes with joblib."""
    num_processes = multiprocessing.cpu_count()
    if num_processes > 68:
        num_processes = 68
    
    # Using joblib's Parallel and delayed to run preprocessing in parallel
    processed_data = Parallel(n_jobs=num_processes)(delayed(preprocess_text)(text) for text in data[column_name])
    return processed_data

# We'll use a predefined list of English stopwords.
stop_words = set(stopwords.words('english'))
stemmer = nltk.stem.PorterStemmer()

if not os.path.isfile('data/preprocessed_queries_train_gpt4.tsv'):
    print('Preprocessing train queries...')
    train_querys = pd.read_csv('data/queries_train_gpt4.csv', sep=',')
    train_querys['processed_query'] = multiprocess_preprocess_joblib(train_querys, 'query')
    train_querys = train_querys.drop(columns=['query'])
    train_querys = train_querys.rename(columns={'processed_query': 'query'})
    train_querys.to_csv('data/preprocessed_queries_train_gpt4.tsv', sep='\t', index=False)
else: 
    print('Loading preprocessed train queries...')
    train_querys = pd.read_csv('data/preprocessed_queries_train_gpt4.tsv', sep='\t')


if not os.path.isfile('data/preprocessed_queries_test_gpt4.tsv'):
    print('Preprocessing test queries...')
    test_querys = pd.read_csv('data/queries_test_gpt4.csv', sep=',')
    test_querys['processed_query'] = multiprocess_preprocess_joblib(test_querys, 'query')
    test_querys = test_querys.drop(columns=['query'])
    test_querys = test_querys.rename(columns={'processed_query': 'query'})
    test_querys.to_csv('data/preprocessed_queries_test_gpt4.tsv', sep='\t', index=False)
else:
    print('Loading preprocessed test queries...')
    test_querys = pd.read_csv('data/preprocessed_queries_test_gpt4.tsv', sep='\t')

# if data/preprocessed_collection.tsv does not exist, create it
if not os.path.isfile('data/preprocessed_collection.tsv'):
    print('Preprocessing collection...')
    dataset = pd.read_csv('data/collection.tsv', sep='\t', names=['id', 'text'])
    dataset['processed_text'] = multiprocess_preprocess_joblib(dataset, 'text')
    dataset = dataset.drop(columns=['text'])
    dataset = dataset.rename(columns={'processed_text': 'text'})
    dataset.to_csv('data/preprocessed_collection.tsv', sep='\t', index=False)
    # dropna but preserve ids
    dataset = dataset.dropna()

else:
    print('Loading preprocessed collection...')
    dataset = pd.read_csv('data/preprocessed_collection.tsv', sep='\t')
    dataset = dataset.dropna()

# index_mapping = get_index_mapping(orig, dataset)


Preprocessing train queries...
Preprocessing test queries...
Loading preprocessed collection...


In [3]:
from rank_bm25 import BM25Okapi
import pickle

# if piccke index is not present load it
if not os.path.isfile('data/bm25_index.pkl'):
    print("Index data not present, creating...")
    bm25 = BM25Okapi(list(dataset['text']))
    with open('data/bm25_index.pkl', 'wb') as handle:
        pickle.dump(bm25, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    print("Loading index data...")
    with open('data/bm25_index.pkl', 'rb') as f:
        bm25 = pickle.load(f)

Loading index data...


In [1]:
from tqdm import tqdm
import heapq

def get_processed_qids(output_filename):
    """Retrieve a set of processed qids from the output file."""
    processed_qids = set()
    try:
        with open(output_filename, 'r') as file:
            for line in file:
                # Extracting the qid from the runfile line format
                processed_qids.add(line.split()[0])
    except FileNotFoundError:
        pass
    return processed_qids

def write_to_runfile(qid, passage_indices, run_identifier, file):
    """Write the results for a single query to the TREC runfile."""
    qid, idxs = passage_indices
    for rank, (passage_idx, score) in enumerate(idxs, 1):
        # Construct the turn identifier from the qid
        topic_id, turn_id = qid.split("_")
        turn_identifier = f"{topic_id}_{turn_id}"

        # Retrieve the document ID from the collection using the passage index
        doc_id = dataset.iloc[passage_idx]['id']

        # Write the formatted line to the file
        file.write(f"{turn_identifier} Q0 {doc_id} {rank} {score} {run_identifier}\n")


# Retrieve top 1000 passages for each query in test_querys
ranking_results_new = {}
top_k = 1000
run_id_new = "BM25"

def retrieve_rankings_score(row):
    qid = row['qid']
    query_text = row['query']

    scores = bm25.get_scores(query_text)

    # Using nlargest to efficiently get top 1000 scores and their indices
    top_scores_indices = heapq.nlargest(1000, enumerate(scores), key=lambda x: x[1])

    # Splitting the (index, score) pairs
    top_indices, top_scores = zip(*top_scores_indices)
    
    return qid, list(zip(top_indices, top_scores))

output_filename_parallel = "data/trec_runfile_434.txt"

# Get the set of already processed qids
processed_qids = get_processed_qids(output_filename_parallel)

with open(output_filename_parallel, 'a') as file:  # Open the file in append mode
    for _, row in tqdm(train_querys.iterrows(), total=train_querys.shape[0]):
        qid = row['qid']
        
        # Check if qid has already been processed
        if qid in processed_qids:
            continue
        
        top_indices = retrieve_rankings_score(row)
        write_to_runfile(qid, top_indices, run_id_new, file)  # Write to file immediately

NameError: name 'train_querys' is not defined